# X First Search
I want to implement some basic search algorithms, and then test them on some toy problems.

**Goals**
1. Readability
2. Generality: I want this to be useable for any relevant problem. Will probably need a few callbacks.
3. Performance: Important, but comes after the other two.

**Looking to implement**
- [Breadth First Search](#DFS-and-BFS)
- [Depth First Search](#DFS-and-BFS)
- [Branch and Bound](#Branch-and-Bound)
- Dijkstra [Coming soon...]

## DFS and BFS
These are basically the same algorithm, but the order you visit the nodes is different.

We'll implement a common `search` function, which will be used for both. It will take a container as an argument to determine the node visiting order.

I want `search` to be general enough that it can be used for any search problem.  
To do this, it will take two callbacks:
- a function which `visit`s a  node, including determining if the search has reached its target
- a function to `get_children` of a node, so that `search` is decoupled from the graphs' implementation

In [1]:
def search(start, visit, get_children, container):
    '''
    start: first node
    visit: when visit(node) is called, it visits it. Only returns a result if the search is finished.
    get_children: when get_children(node) is called, it returns and iterable of nodes to add to the container
    container: Queue (BFS) or Stack (DFS)
    '''
    container.put(start)
    while not container.empty():
        node = container.get()
        
        if (result := visit(node)) is not None:
            return result
        
        for child in get_children(node):
            container.put(child)

[![Walruses make people mad](https://media.giphy.com/media/e9MwdstmbYrhC/giphy.gif "It's my first time, okay!")](https://youtu.be/6uAvHOKofws?t=240)

`search` works by `put`ting each node's children into the container, and then `get`ing them back later in the order you want to `visit` them.

**DFS**  
visits each node's **children before peers**, so we should always `get` the **last** item `put` into the container. i.e. Last In First Out.  
Container should be a [**Stack**](<https://en.wikipedia.org/wiki/Stack_(abstract_data_type)>) \([queue.LifoQueue](https://docs.python.org/3/library/queue.html#queue.LifoQueue))

**BFS**  
visits each node's **peers before children**, so we should always `get` the **first** item `put` into the container. i.e. First In First Out  
Container should be a [**Queue**](<https://en.wikipedia.org/wiki/Queue_(abstract_data_type)>)
\([queue.Queue](https://docs.python.org/3/library/queue.html#queue.Queue))

In [2]:
from queue import Queue, LifoQueue as Stack

def dfs(start, visit, get_children):
    return search(start, visit, get_children, container=Stack())

def bfs(start, visit, get_children):
    return search(start, visit, get_children, container=Queue())

Okay, seems fairly straightforward so far.

## Let's eat!

Let's look at an example implementation using the appetizers problem from [XKCD NP-Complete](https://xkcd.com/287/):

![NP-Complete](https://imgs.xkcd.com/comics/np_complete.png "NP-Complete")

This is a [combinatorial optimisation](https://en.wikipedia.org/wiki/Combinatorial_optimization) problem because we want to find _combinations_ of menu items that will sum to the target price.  
This is exactly the type of problem that BFS and DFS can solve.  
What a coincidence!


In [3]:
# Set up the problem structure

from collections import namedtuple, Counter
from decimal import Decimal

Item = namedtuple('Item', ('name', 'price'))
Node = namedtuple('Node', ('item', 'total', 'prior'))

def dollars(val):
    '''
    Prevents a bunch of floating point errors I was getting.
    Keeps all amounts to the nearest cent.
    '''
    return Decimal(val).quantize(Decimal('0.01')) 

def read(the_menu):
    '''So the_menu can just be a string'''
    menu_lines = (
        line.strip().split() 
        for line in the_menu.splitlines()
        if line
    )
    return {Item(name, dollars(price)) for name, price in menu_lines}

class Order(Counter):
    '''Convenient way to output and display results'''  
    @property
    def total(self):
        return sum(item.price * qty for item, qty in self.items())
    
    def __repr__(self):
        items = '\n'.join(
            f'{qty} x {item.name} @ {item.price} = {qty * item.price}'
            for item, qty in self.items()
        )
        total = f'            Total = {self.total}'
        return f'{items}\n{total}'

In [4]:
# Load the data for this specific problem instance

menu = read(the_menu='''
    fruit 2.15
    fries 2.75
    salad 3.35
    wings 3.55
    sticks 4.20
    plate 5.80
''')


# Check that the waiter understands my order

ill_have_one_of_everything_thanks = Order(menu)
assert ill_have_one_of_everything_thanks.total == dollars(21.80)
ill_have_one_of_everything_thanks

1 x fries @ 2.75 = 2.75
1 x sticks @ 4.20 = 4.20
1 x wings @ 3.55 = 3.55
1 x plate @ 5.80 = 5.80
1 x fruit @ 2.15 = 2.15
1 x salad @ 3.35 = 3.35
            Total = 21.80

Let's define the functions we need for this specific search problem:

**`get_children`** is interesting for this problem because you do not predifine each Node's children. 
- At each Node, the children are the whole menu!  
- Notice that I create a `Node` from each `Item` so that I can keep track of the total and the previous items bought.

**`visit`** is fairly simple - it just checks if we have we have the correct total. 
- If we do, it returns all the items we visited to get to that total.
- I use a [closure](https://en.wikipedia.org/wiki/Closure_(computer_programming)) `visitor` to create the `visit` function because I don't want to hard code the target value, and I can't see a better way to make it available to `visit`

In [5]:
def get_children(node):
    return {
        Node(item=item, total=node.total + item.price, prior=node)
        for item in menu
    }

def get_result(node):
    items = [node.item]
    while node.prior:
        node = node.prior
        if node.item:
            items.append(node.item)
    return Order(items)
    
def visitor(target):
    def visit(node):
        if node.total == target:
            return get_result(node)
    return visit

In [6]:
# Test BFS
target = dollars(15.05)

order = bfs(
    start=Node(item=None, total=0, prior=None),
    visit=visitor(target),
    get_children=get_children,
)

assert order.total == target; order

1 x fruit @ 2.15 = 2.15
2 x wings @ 3.55 = 7.10
1 x plate @ 5.80 = 5.80
            Total = 15.05

**Problem:**  
Since there's no limit on how many appetizers we buy, DFS will run forever since there is no "deepest" point.  
Maybe there's a way to help it stop?

## Branch and Bound
**Concept**  
Once we have bought appetizers exceeding the target price, there's no point to continue exploring that branch, since no matter how much more you buy, you will never get (back down) to the target.

So Branch and Bound is really just a clever way to do (much) less searching. The idea is obvious to a human, but computers don't know unless you tell them!

**Implementation**  
`search()` is basically the same, but expects `visit()` to raise a `PassedBound` exception if it passes a bound and wishes to stop early.

This means it is 100% backwards compatible with the old `search()`, so let's just replace it.

In [7]:
class PassedBound(Exception):
    pass

def search(start, visit, get_children, container):
    '''
    start: first node
    visit: when visit(node) is called, it visits it. Only returns a result if the search is finished.
            Raises a PassedBound exception to prune a branch if we want Branch and Bound.
    get_children: when get_children(node) is called, it returns and iterable of nodes to add to the container
    container: Queue (BFS) or Stack (DFS)
    '''
    container.put(start)
    while not container.empty():
        node = container.get()
        
        try:
            result = visit(node)
        except PassedBound:
            # Don't keep exploring this branch
            continue
            
        if result is not None:
            return result

        for child in get_children(node):
            container.put(child)

In [8]:
def dfs(start, visit, get_children):
    return search(start, visit, get_children, container=Stack())

def bfs(start, visit, get_children):
    return search(start, visit, get_children, container=Queue())

**Update `visit` for Branch and Bound**  
To use Branch and Bound in any `search()`, `visit()` just needs to raise a `PassedBound` exception whenever it wants to prune.  
For the Appetizers problem, it means we will raise `PassedBound` whenever the total price of the products considered exceeds the target price.

In [9]:
def visitor(target):
    def visit(node):
        total = node.total
        if total == target:
            return get_result(node)
        elif total > target:
            raise PassedBound()   
    return visit

In [10]:
# Test DFS

target = dollars(15.05)

order = dfs(
    start=Node(item=None, total=0, prior=None),
    visit=visitor(target),
    get_children=get_children,
)

assert order.total == target; order

2 x wings @ 3.55 = 7.10
1 x plate @ 5.80 = 5.80
1 x fruit @ 2.15 = 2.15
            Total = 15.05

In [11]:
# Test BFS

target = dollars(15.05)

order = bfs(
    start=Node(item=None, total=0, prior=None),
    visit=visitor(target),
    get_children=get_children,
)

assert order.total == target; order

1 x fruit @ 2.15 = 2.15
2 x wings @ 3.55 = 7.10
1 x plate @ 5.80 = 5.80
            Total = 15.05

So far so good :)

## Performance

In [12]:
%%time
target = dollars(28.15)

order = bfs(
    start=Node(item=None, total=0, prior=None),
    visit=visitor(target),
    get_children=get_children,
)

assert order.total == target; order

CPU times: user 1.82 s, sys: 17.1 ms, total: 1.83 s
Wall time: 1.84 s


1 x fries @ 2.75 = 2.75
2 x salad @ 3.35 = 6.70
2 x wings @ 3.55 = 7.10
2 x plate @ 5.80 = 11.60
            Total = 28.15

In [13]:
%%time
target = dollars(15050.00)

order = dfs(
    start=Node(item=None, total=0, prior=None),
    visit=visitor(target),
    get_children=get_children,
)

assert order.total == target; order

CPU times: user 2.58 s, sys: 8.62 ms, total: 2.59 s
Wall time: 2.6 s


842 x fruit @ 2.15 = 1810.30
587 x plate @ 5.80 = 3404.60
571 x salad @ 3.35 = 1912.85
950 x wings @ 3.55 = 3372.50
761 x fries @ 2.75 = 2092.75
585 x sticks @ 4.20 = 2457.00
            Total = 15050.00

**BFS**  
I ran BFS with quite a few different targets, and the run time was really really variable depending on the target.
- As soon as you have more than 6 or 7 items in the order, it takes forever.  
- I think it ends up creating a lot of branches before it starts pruning, whereas DFS explores a single branch then prunes, and keeps doing this.

**DFS**  
DFS seems much better suited to this problem. An order with a few thousand items just takes a few seconds

## Shortest Path
Coming soon...